In [ ]:

import os.path
import os
from pprint import pprint
import boto3
import time
import sys
from dotenv import load_dotenv
load_dotenv()
import requests
import json
import pandas as pd
import subprocess

from time import gmtime, strftime
from datetime import datetime
import dateutil

import datetime

import tscribe
from pprint import pprint

import requests
import json
import pandas as pd
import subprocess

from time import gmtime, strftime
from datetime import datetime
import dateutil
import csv
import shutil
import matplotlib.pyplot as plt
import datetime

In [ ]:
import numpy as np
import re
from nltk import tokenize

from scipy.interpolate import make_interp_spline, BSpline
from matplotlib.ticker import MaxNLocator
from scipy.interpolate import UnivariateSpline
import pickle
import nltk
from datetime import datetime
from datetime import timedelta
import string
import random

nltk.download('punkt')

EXT = 'mp4'
os.environ['MKL_THREADING_LAYER'] = 'GNU'

CSV = False

#performance
TIMES = []


#6. Transcribe
def transcribe(bucket_name, ad_filename):
    #step #1 runs transcriber
    session = boto3.Session(os.getenv("A"),os.getenv("B"))
    transcribe = session.client('transcribe', region_name='us-east-1')
    job_name = "abc" #ignore
    job_uri = 's3://' + str(bucket_name) + '/'

    files = [ad_filename]

In [ ]:
   #TODO filename has no '.'
    file_name = ad_filename.split('.')[0]
    EXT = ad_filename.split('.')[-1]
    new_file_name = file_name + "." + "mp4"

    for item in files:
        if not CSV:
            start_tsc = time.time()
            job_uri = job_uri + item
            job_name =  str(int(time.time()))
            transcribe.start_transcription_job(
                    TranscriptionJobName=job_name,
                    Media={'MediaFileUri': job_uri},
                    MediaFormat= 'wav',
                    LanguageCode= 'en-US',
                    Settings={
                        'ChannelIdentification': False,
                        "MaxAlternatives": 2,
                        "MaxSpeakerLabels": 2,
                        "ShowAlternatives": True,
                        "ShowSpeakerLabels": True}
                    )
            print("Transcribing..")
            s = time.time()
            while True:
                status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
                if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
                    break
                print("Not ready yet...")
                time.sleep(5)
            e =  time.time()

            print("[INFO] Got answer from server in {}".format(e-s))
            url = str(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
            headers = {'Accept': 'application/json'}
            response = requests.get(url, headers=headers)
            print("[INFO] Got file asrout.")
            with open(os.path.splitext(item)[0] + ".json", 'wb') as outf:
                outf.write(response.content)
            tscribe.write(os.path.splitext(item)[0] + ".json", format="csv", save_as=os.path.splitext(item)[0] + ".csv")
            print("[INFO] CSV file available with time stamps.")
            end_tsc = time.time()
            TIMES.append(("TRANSCRIPTION",end_tsc-start_tsc))

In [ ]:
  #TIMES.append(("TRANSCRIPTION",end_tsc-start_tsc))
            df = pd.read_csv(os.path.splitext(item)[0] + ".csv")
        else:
            print("[INFO] CSV file already exist for this audio file.")
            df = pd.read_csv(os.path.splitext(item)[0] + ".csv")

#expr NOTE in this one time is added and _wt arrays are created for the next step
#------------------------------------------------------------------------
        #step 2: questions
        speaker = 'spk_0'
        final_list_wt = []
        final_list_strings_wt = []
        final_speaker_list_wt = []

        #TODO extend this for >2 speakers
        def flip_speaker(speaker):
            if speaker == "spk_0":
                speaker = "spk_1"
            else:
                speaker = "spk_0"
            return speaker

        xs = []
        ys = []
        spk = []
        minute = 0
        overall_co = 0
        overall_cli = 0
        for i,item in df.iterrows():
            #Add this for graphs
            st = time.strptime(str(item['start_time']),'%H:%M:%S')
            et = time.strptime(str(item['end_time']),'%H:%M:%S')
            s = timedelta(hours=st.tm_hour,minutes=st.tm_min,seconds=st.tm_sec).total_seconds()
            e = timedelta(hours=et.tm_hour,minutes=et.tm_min,seconds=et.tm_sec).total_seconds()
            xs.append((e-s)/2 + minute)
            minute = (e-s)/2 + minute
            ys.append(len(item['comment']))
            if item['speaker'] == 'spk_1':
                spk.append('b')
                overall_co = overall_co + (e-s)
            else:
                spk.append('r')
                overall_cli = overall_cli + (e-s)


In [ ]:
  overall_cli = overall_cli + (e-s)
            if final_list_wt:
                last = final_list_wt.pop(-1)
            else:
                last = []
            if item['speaker'] == speaker:
                last.append((item['comment'], (s,e)))
                final_list_wt.append(last)
            elif (item['speaker'] != speaker and len(item['comment']) < 5):
                last.append((item['comment'], (s,e)))
                final_list_wt.append(last)
            else:
                final_list_wt.append(last)
                l = [(item['comment'], (s,e))]
                final_list_wt.append(l)
                speaker = item['speaker']

        #Accumulate times
        speaker = 'spk_0'
        for item in final_list_wt:
            fs = ''
            ts = 0
            sx = 0
            ex = 0
            prev = 0
            if item:
                final_speaker_list_wt.append(speaker)
                speaker = flip_speaker(speaker)
                (_, (sx,_))= item[0]
                (_, (_,ex)) = item[-1]
                for (s, (st,et)) in item:
                    fs = fs + ' ' + s
                    ts = ts + (et-st)
                    if st != prev:
                        ts += 1 #NOTE every segment in same speaker misses 1 second!
                    prev = et
                final_list_strings_wt.append((fs,ts, (sx,ex)))
            else:
                continue

        #NOTE identify the coach/therapist based on questions
        all_spk_0 = 0
        all_spk_1 = 0

In [ ]:
    for idx, (item,ts,(sx,ex)) in enumerate(final_list_strings_wt):
            if final_speaker_list_wt[idx] == 'spk_0' and '?' in item:
                all_spk_0 += 1
            elif final_speaker_list_wt[idx] == 'spk_1' and '?' in item:
                all_spk_1 += 1
        if all_spk_1 > all_spk_0:
            speaker_coach = 'spk_1'
            speaker_client = 'spk_0'
        else:
            speaker_coach = 'spk_0'
            speaker_client = 'spk_1'

        speaker_coach = 'spk_0'
        speaker_client = 'spk_1'

        start_an = time.time()
        #NOTE before getting to questions and answers
        xs_wt = []
        ys_wt = []
        ys_se = []
        ys_pers = []
        spk = []
        qs_wt = [] #TODO
        qs_wft = []
        #TODO extend this to search properly
        def find_next_answer(idx):
            for ix, item in enumerate(final_list_strings_wt):
                if ix == idx:
                    if final_speaker_list_wt[ix+1] == speaker_client :
                        print("Next answer is -----> {}\n".format(final_list_strings_wt[ix+1]))
                        return final_list_strings_wt[ix+1]
                    else:
                        if final_speaker_list_wt[ix+2] == speaker_client:
                            print("Next answer is ------> {}\n".format(final_list_strings_wt[ix+2]))
                            return final_list_strings_wt[ix+2]
                        else:
                            print("\n------ No next answer found!! ------\n")
                            return None

        list_of_questions_answers = []
        for idx, (item,ts,(sx,ex)) in enumerate(final_list_strings_wt):
            if  '?' in item  and final_speaker_list_wt[idx] == speaker_coach:
                q, _, (_,_)= final_list_strings_wt[idx]
                list_of_questions_answers.append(q)
                print("Question is ------> {}\n".format(final_list_strings_wt[idx]))

In [ ]:
       #Time stamps for questions
                min_start = int(sx/60)
                sec_start = int(sx - min_start*60)
                min_end = int(ex/60)
                sec_end = int(ex - min_end*60)
                qs_wt.append("[start: {}:{} end: {}:{}]".format(min_start, sec_start, min_end, sec_end))
                qs_wft.append(sx/60)
                try:
                    (answ, ats, (asx, aex)) = find_next_answer(idx)
                    print(answ)
                except:
                    print("[INFO] No next answer found!")
                    ys_pers.append(0.1)
                    xs_wt.append(item)
                    ys_wt.append(0.1)
                    ys_se.append(0.1)
                    aex = ex
                    continue
                overall = aex - sx
                ys_pers.append(int(((ex- sx)/overall)*100))
                xs_wt.append(item)
                ys_wt.append(aex - ex)
                ys_se.append(ats)

        #EXPR NOTE
        all_the_text = []
        for idx, (item,ts,(sx,ex)) in enumerate(final_list_strings_wt):
            all_the_text.append(item)
       # print(' '.join(all_the_text))

#=================================================================================


In [ ]:
       num_of_questions = len(xs_wt)

        print("=============================================Questions===============================================")
        questions_numbered = []
        questions_timed = []
        for idx,item in enumerate(xs_wt):
            questions_numbered.append("    {}. {} {}".format(idx, qs_wt[idx], item))
            questions_timed.append((item, qs_wft[idx]))
        list_of_questions = '\n'.join(questions_numbered)
        return

bucket_name = 'user-mashoumi'
ad_filename = 'therapy2.wav'

transcribe(bucket_name, ad_filename)
